<a href="https://colab.research.google.com/gist/parulnith/7f8c174e6ac099e86f0495d3d9a4c01e/untitled9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Music genre classification notebook

## Importing Libraries

In [1]:
# feature extractoring and preprocessing data
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

#Keras
import keras

import warnings
warnings.filterwarnings('ignore')

C:\Users\Jagpreet singh\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Extracting music and features

### Dataset

We use [GTZAN genre collection](http://marsyasweb.appspot.com/download/data_sets/) dataset for classification. 
<br>
<br>
The dataset consists of 10 genres i.e
 * Blues
 * Classical
 * Country
 * Disco
 * Hiphop
 * Jazz
 * Metal
 * Pop
 * Reggae
 * Rock
 
Each genre contains 100 songs. Total dataset: 1000 songs

## Extracting the Spectrogram for every Audio

In [2]:
cmap = plt.get_cmap('inferno')

plt.figure(figsize=(10,10))
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    pathlib.Path(f'img_data/{g}').mkdir(parents=True, exist_ok=True)     
    for filename in os.listdir(f'./MIR/genres/{g}'):
        songname = f'./MIR/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=5)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'img_data/{g}/{filename[:-3].replace(".", "")}.png')
        plt.clf()
 

<Figure size 720x720 with 0 Axes>

All the audio files get converted into their respective spectrograms .We can now easily extract features from them.

## Extracting features from Spectrogram


We will extract

* Mel-frequency cepstral coefficients (MFCC)(20 in number)
* Spectral Centroid,
* Zero Crossing Rate
* Chroma Frequencies
* Spectral Roll-off.

In [3]:
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

## Writing data to csv file

We write the data to a csv file 

In [4]:
file = open('data.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    for filename in os.listdir(f'./MIR/genres/{g}'):
        songname = f'./MIR/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        rmse=librosa.feature.rmse(y=y)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('data.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

The data has been extracted into a [data.csv](https://github.com/parulnith/Music-Genre-Classification-with-Python/blob/master/data.csv) file.

# Analysing the Data in Pandas

In [5]:
data = pd.read_csv('data.csv')
data.head()

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,blues.00000.au,0.349943,0.130225,1784.420446,2002.650192,3806.485316,0.083066,-113.596742,121.557302,-19.158825,...,8.810668,-3.667367,5.751690,-5.162761,0.750947,-1.691937,-0.409954,-2.300208,1.219928,blues
1,blues.00001.au,0.340983,0.095918,1529.835316,2038.617579,3548.820207,0.056044,-207.556796,124.006717,8.930562,...,5.376802,-2.239119,4.216963,-6.012273,0.936109,-0.716537,0.293875,-0.287431,0.531573,blues
2,blues.00002.au,0.363603,0.175573,1552.481958,1747.165985,3040.514948,0.076301,-90.754394,140.459907,-29.109965,...,5.789265,-8.905224,-1.083720,-9.218359,2.455805,-7.726901,-1.815724,-3.433434,-2.226821,blues
3,blues.00003.au,0.404779,0.141191,1070.119953,1596.333948,2185.028454,0.033309,-199.431144,150.099218,5.647594,...,6.087676,-2.476420,-1.073890,-2.874777,0.780976,-3.316932,0.637981,-0.619690,-3.408233,blues
4,blues.00004.au,0.308590,0.091563,1835.494603,1748.362448,3580.945013,0.101500,-160.266031,126.198800,-35.605448,...,-2.806385,-6.934122,-7.558619,-9.173552,-4.512166,-5.453538,-0.924162,-4.409333,-11.703781,blues


In [6]:
data.shape

(1000, 28)

In [7]:
# Dropping unneccesary columns
data = data.drop(['filename'],axis=1)

## Encoding the Labels

In [8]:
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)

## Scaling the Feature columns

In [9]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))

## Dividing data into training and Testing set

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [11]:
len(y_train)

800

In [12]:
len(y_test)

200

In [13]:
X_train[10]

array([-0.86631738,  0.0130515 , -1.04817865, -1.26444028, -1.02691956,
       -1.00929623, -0.34851614,  1.17852756, -0.81785072,  0.22180453,
        0.52685605, -0.31345107, -1.55306262, -0.15356476, -0.52670732,
       -1.41156016, -0.35116952, -1.70189325,  0.20310153, -1.96381491,
       -1.04459693, -2.09147952, -1.44918905, -2.92087238, -0.63003752,
       -1.96674746])

# Classification with Keras

## Building our Network

In [14]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))

model.add(layers.Dense(128, activation='relu'))

model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(10, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [15]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [16]:
history = model.fit(X_train,
                    y_train,
                    epochs=20,
                    batch_size=128)
                   

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
800/800 [==============================] - 6s 8ms/step - loss: 2.1668 - acc: 0.2475
Epoch 2/20
800/800 [==============================] - 0s 29us/step - loss: 1.8518 - acc: 0.3275
Epoch 3/20
800/800 [==============================] - 0s 34us/step - loss: 1.6428 - acc: 0.3725
Epoch 4/20
800/800 [==============================] - 0s 24us/step - loss: 1.4742 - acc: 0.4600
Epoch 5/20
800/800 [==============================] - 0s 25us/step - loss: 1.3354 - acc: 0.5162
Epoch 6/20
800/800 [==============================] - 0s 25us/step - loss: 1.2258 - acc: 0.5988
Epoch 7/20
800/800 [==============================] - 0s 26us/step - loss: 1.1374 - acc: 0.6325
Epoch 8/20
800/800 [==============================] - 0s 24us/step - loss: 1.0683 - acc: 0.6600
Epoch 9/20
800/800 [==============================] - 0s 24us/step - loss: 0.9950 - acc: 0.6750
Epoch 10/20
800/800 [==============================] - 0s 26us/step - loss: 0.9483 - acc:

In [17]:
test_loss, test_acc = model.evaluate(X_test,y_test)

200/200 [==============================] - 0s 260us/step


In [18]:
print('test_acc: ',test_acc)

test_acc:  0.71


Test accuracy is less than training dataa accuracy. This hints at Overfitting

## Validating our approach
Let's set apart 200 samples in our training data to use as a validation set:

In [19]:
x_val = X_train[:200]
partial_x_train = X_train[200:]

y_val = y_train[:200]
partial_y_train = y_train[200:]

Now let's train our network for 20 epochs:

In [20]:
model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(partial_x_train,
          partial_y_train,
          epochs=20,
          batch_size=512,
          validation_data=(x_val, y_val))
results = model.evaluate(X_test, y_test)

Train on 600 samples, validate on 200 samples
Epoch 1/20
600/600 [==============================] - 0s 663us/step - loss: 2.3097 - acc: 0.1017 - val_loss: 2.2085 - val_acc: 0.2450
Epoch 2/20
600/600 [==============================] - 0s 53us/step - loss: 2.1632 - acc: 0.3317 - val_loss: 2.0922 - val_acc: 0.3250
Epoch 3/20
600/600 [==============================] - 0s 45us/step - loss: 2.0259 - acc: 0.4000 - val_loss: 1.9834 - val_acc: 0.3700
Epoch 4/20
600/600 [==============================] - 0s 43us/step - loss: 1.8821 - acc: 0.4183 - val_loss: 1.9049 - val_acc: 0.3800
Epoch 5/20
600/600 [==============================] - 0s 53us/step - loss: 1.7586 - acc: 0.4150 - val_loss: 1.8407 - val_acc: 0.3900
Epoch 6/20
600/600 [==============================] - 0s 47us/step - loss: 1.6387 - acc: 0.4333 - val_loss: 1.7693 - val_acc: 0.4050
Epoch 7/20
600/600 [==============================] - 0s 47us/step - loss: 1.5293 - acc: 0.5050 - val_loss: 1.6716 - val_acc: 0.4300
Epoch 8/20
600/600 [==

In [21]:
results

[1.1203268313407897, 0.655]

## Predictions on Test Data

In [22]:
predictions = model.predict(X_test)

In [23]:
predictions[0].shape

(10,)

In [24]:
np.sum(predictions[0])

0.99999994

In [25]:
np.argmax(predictions[0])

2